# Simplex algorithm

## Understanding the algorithm

The simplex algorithm is a mathematical method for solving linear programming problems. It was developed by George Dantzig in 1947. The basic principle of linear programming is to maximize or minimize a linear function, called the objective function, under linear constraints as well.

Used for continuous linear optimization problems.

Here are the main steps of the simplex algorithm:

1. **Problem Formulation**: The problem is formulated in terms of an objective function to be maximized or minimized. For example, maximizing profit or minimizing cost.

2. **Linear Constraints**: The constraints of the problem are expressed as linear equations or inequalities.

3. **Feasible Basic Solution (FBS)**: An initial feasible basic solution is identified. An FBS is a solution that satisfies all the constraints and where the number of non-zero variables does not exceed the number of constraints.

4. **Optimality Evaluation**: It is checked whether the current FBS maximizes (or minimizes) the objective function. If this is the case, the solution is optimal.

5. **Iteration Towards a Better Solution**: If the current solution is not optimal, the algorithm performs a pivot. This involves choosing an entry variable (to increase the value of the objective function) and an exit variable (to maintain the constraints), and then recalculating the FBS.

6. **Termination**: The algorithm ends when the optimal solution is reached or when there is no better possible solution (for example, in the case of an unbounded problem).

## Usage examples

The simplex algorithm is used in many situations where it is necessary to solve linear programming problems, that is, problems where one seeks to optimize (maximize or minimize) a linear function subject to linear constraints as well. Here are some typical contexts of use:

1. **Resource Optimization**: In the industrial or production sector, for efficiently allocating limited resources (such as raw materials, machine time, or personnel) to maximize profit or productivity, or to minimize costs.

2. **Planning and Scheduling**: For developing optimal schedules, whether in project management, flight scheduling in airlines, or production planning in factories.

3. **Portfolio Management in Finance**: For the optimal selection of investments or the diversification of portfolios, where the goal is to maximize expected returns while managing risk.

4. **Logistics and Transport**: To optimize delivery routes, inventory management, or vehicle assignment, in order to reduce costs or improve logistic efficiency.

5. **Network and Telecommunications**: For optimal management of network traffic, bandwidth allocation, or network capacity planning.

6. **Food and Agricultural Optimization**: To maximize crop yields or the efficiency of diets, considering nutritional constraints or available resources.

7. **Operations Research and Strategic Decision Making**: In strategic decision-making for businesses or organizations, where various factors and constraints must be balanced to achieve an optimal overall objective.

8. **Healthcare Sector**: For managing medical staff schedules, planning patients for various procedures, or managing hospital resources.

___

## Strengths

1. **Efficiency for Many Problems**: Although theoretically the simplex can take exponential time, in practice, it is very efficient for many real-world problems.

2. **Guaranteed Optimal Solution**: If an optimal solution exists, the simplex will find it. It is precise and reliable in finding an optimal solution.

3. **Flexibility and Adaptability**: The simplex can handle changes in the constraints or the objective function without having to start from scratch.

4. **Interpretable Information**: The algorithm not only provides an optimal solution but also information about the sensitivity of solutions to changes in the problem's parameters.

5. **Wide Applicability**: It can be applied to a wide range of problems in different fields such as economics, logistics, resource management, etc.

---

## Weaknesses

1. **Complexity with Large Data Sets**: For very large-scale problems or with a large number of constraints, the simplex can become inefficient in terms of computation time.

2. **Unbounded or Tricky Problems**: In some cases, such as unbounded problems or those with degenerate solution regions, the simplex may not perform optimally.

3. **Lack of Efficiency in Parallelization**: Unlike other methods, the simplex algorithm is difficult to parallelize efficiently, limiting its speed on modern computer systems.

4. **Sensitivity to Rounding Errors**: As it relies on numerical calculations, it can be sensitive to rounding errors, especially in cases where the equations are nearly degenerate.

5. **Not Universally Applicable**: The simplex only applies to linear programming problems and cannot be used for non-linear or discrete problems (such as integer programming).
___

## Python demonstration

### Problem:
- **Products**: Shoes, T-shirts, Jeans, Socks


- **Profit per Unit**:
  - Shoes: $200
  - T-shirts: $45
  - Jeans: $60
  - Socks: $15
- **Required Resources per Unit** (in resource units):
  - Shoes: 5 of leather, 2 of plastic, 1 of cotton
  - T-shirts: 3 of leather, 3 of plastic, 2 of cotton
  - Jeans: 4 of leather, 0 of plastic, 3 of cotton
  - Socks: 2 of leather, 2 of plastic, 4 of cotton
- **Available Resources**:
  - Leather: 100 units
  - Plastic: 90 units
  - Cotton: 120 units

### Objective:
Maximize total profit.

### Constraints:
1. The use of resources must not exceed the available quantities.
2. The quantities of each product must be positive or zero.

### Notes:
- The coefficients in `c` represent the profits per unit for each product.
- The matrix `A` and the vector `b` represent the constraints related to resource utilization.
- In a real scenario, you would also need to manage aspects such as the selection of the entering and leaving variables, degenerate cases, and unbounded solutions in the implementation of the simplex.

In [20]:
import numpy as np

def simplex(A, b, c):
    nrows, ncols = A.shape
    X = np.zeros(ncols)
    basis = [False] * ncols

    basis[-nrows:] = [True] * nrows
    B_inv = np.linalg.inv(A[:, basis])

    while True:
        X[basis] = np.dot(B_inv, b)

        # Assurer que les quantités sont positives ou nulles
        X = np.maximum(X, 0)

        reduced_costs = c - np.dot(np.dot(c[basis], B_inv), A)
        if all(reduced_cost >= 0 for reduced_cost in reduced_costs):
            # Vérifier à nouveau les quantités avant de retourner le résultat
            rounded_solution = np.maximum(np.round(X), 0)
            return rounded_solution

        entering = np.where(reduced_costs < 0)[0][0]
        delta = np.dot(B_inv, A[:, entering])
        if all(d <= 0 for d in delta):
            raise Exception("Problem is unbounded.")

        ratios = [X[basis_var]/delta_var for basis_var, delta_var in zip(range(ncols), delta) if delta_var > 0]
        leaving = np.where(basis)[0][np.argmin(ratios)]

        basis[leaving] = False
        basis[entering] = True
        B_inv = np.linalg.inv(A[:, basis])

# Define product names corresponding to variables
product_names = ["Shoes", "T-shirts", "Jeans", "Socks"]

# Define the coefficients for the objective function (profits)
c = np.array([200, 45, 60, 15])

# Define the matrix of coefficients for the constraints
A = np.array([
# the quantities and materials are not intended to be realistic
    [5, 3, 4, 2],  # Leather, for example, you need 5 units of leather to make shoes while uou need 2 units to maje a jean.
    [2, 3, 0, 2],  # Plastic
    [1, 2, 3, 4]   # Cotton
])

# Define the right-hand side of the constraints (resource availability)
b = np.array([100, 90, 120])

# Solve the linear programming problem
optimal_values = simplex(A, b, -c)  # Negate c for maximization

# Associate optimal values with product names 
optimal_quantities = dict(zip(product_names, optimal_values))

# Associate price with product names 
product_prices = dict(zip(product_names, c))
total_earned = 0

print("Optimal quantities for each products:")
for product, quantity in optimal_quantities.items():
    price = product_prices[product]

    total_earned += quantity * price
    print(f"{product}: {quantity}")

print(f"Total earned: ${total_earned}")

Optimal quantities for each products:
Shoes: 13.0
T-shirts: 17.0
Jeans: 0.0
Socks: 32.0
Total earned: $3845.0


🚨 **Attention: Independent Constraints Required**

<div class="alert alert-block alert-danger">
When using the Simplex algorithm for linear programming, ensure your constraints (matrix `A`) are independent. A singular (non-invertible) matrix, caused by dependent constraints, leads to computational errors. Check this by ensuring the matrix's rank equals its number of columns. Adjust your constraints if necessary to avoid singularity issues.
</div>

### Portfolio problem resolution with simplex

In [21]:
import pulp
from assets.datasets.portfolio import Portfolio

portfolio = Portfolio(10) # Select 10 companies
assets = portfolio.get_assets()

In [22]:
# Create a linear programming problem
prob = pulp.LpProblem("PortfolioOptimization", pulp.LpMaximize)

# Variables for the number of stocks purchased (integers)
stock_quantities = pulp.LpVariable.dicts("Stocks", [a['AssetName'] for a in assets], 0, cat=pulp.LpInteger)

# Risk aversion coefficient
risk_aversion = 1

# Total budget for investment
total_budget = 30000

# Objective function: Maximize risk-adjusted return
prob += pulp.lpSum([(a['ExpectedReturn'] - risk_aversion * a['Risk']) * stock_quantities[a['AssetName']]
                    for a in assets]), "TotalExpectedReturn"

# Constraint: Total investment value should not exceed the budget
prob += pulp.lpSum([a['UnitCost'] * stock_quantities[a['AssetName']] for a in assets]) <= total_budget, "BudgetConstraint"

# Solve the problem
prob.solve(pulp.apis.PULP_CBC_CMD(msg=False)) # Log message disabled

# Display results
print("Optimal Portfolio: \n")
total_spent = 0
total_expected_return = 0
for asset in assets:
    asset_name = asset['AssetName']
    quantity = stock_quantities[asset_name].varValue
    total_spent += asset['UnitCost'] * quantity
    total_expected_return += quantity * asset['UnitCost'] * asset['ExpectedReturn'] * 0.01
    print(f"{asset_name}: {quantity}")

expected_return_percentage = (total_expected_return * 100) / total_spent

print(f"\n Total spent: {total_spent},\n Expected return: {total_expected_return} ({expected_return_percentage}%)")


Optimal Portfolio: 

Nebuchadnezzar Shipping: 0.0
Shinra Electric Power Company: 0.0
VersaLife: 0.0
Sterling Cooper: 0.0
Capsule Corp: 171.0
The Chum Bucket: 1.0
Umbrella Corporation: 0.0
Paper Street Soap Company: 0.0
Planetary Express: 0.0
Primatech Paper Co: 0.0

 Total spent: 30000.0,
 Expected return: 6020.325 (20.06775%)


---
## Practical optimization tools

1. **SciPy**:
   - **Library**: SciPy (`scipy.optimize.linprog`)
   - **Features**: Part of the larger SciPy ecosystem, it offers a high-level interface for linear programming, including the Simplex method.
   - **Usage**: Suitable for a wide range of applications, from academic research to industrial problems.
   - **Ease of Use**: It is user-friendly, especially for those already familiar with the SciPy stack.

2. **PuLP**:
   - **Library**: PuLP
   - **Features**: An LP modeler written in Python. PuLP can call multiple solvers, including its own implementation of the Simplex algorithm.
   - **Usage**: Good for formulating LP problems in a more Pythonic way.
   - **Ease of Use**: Very intuitive for Python users, allows easy model creation and solution extraction.

3. **CVXOPT**:
   - **Library**: CVXOPT
   - **Features**: A Python library for convex optimization. It includes utilities for solving linear programming problems.
   - **Usage**: More suitable for convex optimization problems but can be used for linear programming as well.
   - **Ease of Use**: A bit more complex than PuLP or SciPy but offers more control and power for complex problems.

4. **Google OR-Tools**:
   - **Library**: Google OR-Tools
   - **Features**: Provides operations research tools, including linear programming with various solvers.
   - **Usage**: Very powerful for a wide range of optimization problems, including routing, scheduling, and planning.
   - **Ease of Use**: Offers both a low-level and a high-level API, suitable for both simple and complex optimization tasks.

5. **Gurobi** and **CPLEX**:
   - **Libraries**: Gurobi and IBM ILOG CPLEX
   - **Features**: Both are commercial optimization solvers with Python APIs, known for their performance and robustness.
   - **Usage**: Widely used in industry for large-scale and complex linear programming problems.
   - **Ease of Use**: Both provide Python interfaces but may have a steeper learning curve and require a license (free academic licenses are often available).

## Sources